# **1. Corpus**

## Crawl URLs to extract all internal links 

**XENU Link Sleuth**  
https://home.snafu.de/tilman/xenulink.html 

*Le logiciel XENU Link Sleuth a finalement été retenu pour cette tâche* 

## Scrape textual data from crawled URLs
**BeautifulSoup HTML Parser**  
Réf : https://realpython.com/python-web-scraping-practical-introduction/

In [15]:
path = '/Users/camilledemers/Documents/03-corpus/0-listes_sites/'
acteur = "gmf"

La liste des URLs à scrapper pour chaque corpus est contenue dans un fichier CSV. 
On commence donc par lire le CSV pour extraire nos URLS.

In [16]:
from pandas import *

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur + "/" + acteur + '.csv', encoding='utf-8') as f:
    csv = read_csv(f, sep=";")

csv

,Address,Status-Code,Status-Text,Type,Size,Title,Date,Level,Links Out,Links In,Server,Error,Duration,Charset,Description
0,http://cliniqueazur.com/,200,ok,text/html,15720.0,Clinique Azur,20.05.2022 01:46:10,0,45.0,16.0,Microsoft-IIS/10.0,NaN,00:00.300,NaN,La Clinique Azur est une urgence (anciennement...
1,http://cliniqueazur.com/Emplois,200,ok,text/html,155.0,Emplois,NaN,1,1.0,1.0,Microsoft-IIS/10.0,NaN,00:00.188,utf-8,NaN
2,http://cliniqueazur.com/Emplois/,200,ok,text/html,5910.0,Demandes d'emplois - Clinique Azur,10.02.2022 23:40:48,1,22.0,2.0,Microsoft-IIS/10.0,NaN,00:00.104,NaN,Demandes d'emploi à la clinique Azur
3,http://cliniqueazur.com/Localisation/,200,ok,text/html,8346.0,Localisation - Clinique Azur,20.02.2022 15:59:03,1,23.0,13.0,Microsoft-IIS/10.0,NaN,00:00.157,NaN,Localisation de la Clinique Azur
4,http://cliniqueazur.com/RendezVous/,200,ok,text/html,8676.0,Rendez-vous - Clinique Azur,20.02.2022 15:57:13,1,36.0,9.0,Microsoft-IIS/10.0,NaN,00:00.186,NaN,Prendre un rendez-vous à la clinique Azur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864,https://www.proximite.co/contact/,200,ok,text/html,54862.0,Contact | Clinique Proximité,NaN,1,56.0,7.0,openresty,NaN,00:00.545,utf-8,NaN
1865,https://www.proximite.co/equipe/,200,ok,text/html,123552.0,Équipe | Clinique Proximité,NaN,1,53.0,7.0,openresty,NaN,00:00.324,utf-8,NaN
1866,https://www.proximite.co/faq/,200,ok,text/html,48394.0,FAQ | Clinique Proximité,NaN,1,53.0,7.0,openresty,NaN,00:00.666,utf-8,NaN
1867,https://www.proximite.co/nouvelles/,200,ok,text/html,44673.0,Nouvelles | Clinique Proximité,NaN,1,53.0,7.0,openresty,NaN,00:00.253,utf-8,NaN


In [17]:
liste = csv['Address'].tolist()
liste

['http://cliniqueazur.com/',
 'http://cliniqueazur.com/Emplois',
 'http://cliniqueazur.com/Emplois/',
 'http://cliniqueazur.com/Localisation/',
 'http://cliniqueazur.com/RendezVous/',
 'http://cliniqueazur.com/RendezVous/CliniqueVoyage/',
 'http://cliniqueazur.com/RendezVous/COVID/',
 'http://cliniqueazur.com/RendezVous/GMF/',
 'http://cliniqueazur.com/RendezVous/PriseSang/',
 'http://cliniqueazur.com/RendezVous/Specialistes/',
 'http://cliniqueazur.com/RendezVous/Urgence/',
 'http://cliniqueazur.com/RendezVous/UrgenceGMF/',
 'http://cliniqueazur.com/Repertoire/',
 'http://cliniquemedicale1851.com/',
 'http://cliniquemednam.com/',
 'http://cliniqueviau.com/',
 'http://cliniqueviau.com/?p=17',
 'http://cliniqueviau.com/?p=18',
 'http://cliniqueviau.com/?p=81',
 'http://cliniqueviau.com/index.php/ou-nous-trouver-find-us-here/',
 'http://cliniqueviau.com/index.php/parternariats/',
 'http://cliniqueviau.com/index.php/services/',
 'http://gmfrtr.ca/',
 'http://medicentrechomedey.com/',
 'ht

In [18]:
import requests, re, ssl, os, sys
from bs4 import BeautifulSoup
#from requests.packages.urllib3.util.retry import Retry
from slugify import slugify

def getTextURL(url):    
    html = requests.get(url, headers = {'User-Agent': 'My User Agent 1.0'}, verify=False)
    html.encoding = 'utf-8'
    html = html.text
   
    soup = BeautifulSoup(html, "html.parser")
    data = soup.get_text(separator=' ').replace("\n", " ").replace("\r", " ").replace(u'\xa0', u' ')
    data = re.compile(r"\s+").sub(" ", data).strip() + "\n\n"
    
    return data

In [19]:
path = '/Users/camilledemers/Documents/03-corpus/2-data/1-fr/'
for site in liste: 
    data = ""
    try: 
        text = getTextURL(site)
        data += text

        # encoding = liste[1][liste[0].index(site)] - pour prendre l'encodage qui a été extrait par le crawleur
        file_path = path + acteur + "/" + acteur + "-" + str(liste.index(site)) + ".txt"
        with open(file_path, 'w', encoding = "utf-8") as f:
            f.write(site + "\n")
            f.write(data)
        
    except Exception as e:
        print("ERROR " + " - " + site)
        print(e)

/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'santemontroyal.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cdsdusommet.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cdsdusommet.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.war

ERROR  - https://www.letraitdunion.ca/?lang=en
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.letraitdunion.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.letraitdunion.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.letraitdunion.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings


ERROR  - https://www.letraitdunion.ca/contact
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.letraitdunion.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.letraitdunion.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.letraitdunion.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings


ERROR  - https://www.letraitdunion.ca/services
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.letraitdunion.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ERROR  - https://www.letraitdunion.ca/services?lang=en
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.letraitdunion.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.letraitdunion.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.letraitdunion.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings


## Combine .txt files into corpora
Réf : https://www.geeksforgeeks.org/how-to-read-multiple-text-files-from-folder-in-python/

Pour comparer avec TermoStat (qui prend un seul fichier)

In [20]:
# Importer les données (fichiers .txt)
# import os

# acteur = "chu"
# path = '/Users/camilledemers/Documents/03-corpus/2-data/1-fr/'
# corpus = []

In [21]:
# Change the directory
# os.chdir(path + acteur + "/")
# len(os.listdir())

# for file in os.listdir():
  #  if file.endswith(".txt"):
   #     file_path = path + acteur + "/" + file
        
    #    with open(file_path, 'r', encoding = "UTF-8") as f:
     #       data = f.readlines()
      #      corpus.append(data[1])

#with open (path + acteur + "/" + acteur + '-corpus_FR.txt', 'w', encoding= 'UTF-8') as f: 
   # for s in corpus:
       # f.write(s) 
#f.close()